In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
# Normalize pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [ ]:
# Reshape the data into 1D arrays
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Define your neural network architecture
def create_model(hidden_layer_size=64, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential([
        Dense(hidden_layer_size, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(dropout_rate),
        Dense(hidden_layer_size, activation='relu'),
        Dropout(dropout_rate),
        Dense(10, activation='softmax')  # 10 classes for Fashion MNIST
    ])

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Define hyperparameters to tune
param_grid = {
    'hidden_layer_size': [64, 128, 256],
    'dropout_rate': [0.2, 0.3, 0.4],
    'learning_rate': [0.001, 0.01, 0.1]
}

In [ ]:
# Perform hyperparameter tuning
best_accuracy = 0
best_params = {}

for hidden_layer_size in param_grid['hidden_layer_size']:
    for dropout_rate in param_grid['dropout_rate']:
        for learning_rate in param_grid['learning_rate']:
            model = create_model(hidden_layer_size=hidden_layer_size, dropout_rate=dropout_rate, learning_rate=learning_rate)
            early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
            history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)
            accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {
                    'hidden_layer_size': hidden_layer_size,
                    'dropout_rate': dropout_rate,
                    'learning_rate': learning_rate
                }


In [ ]:
print("Best Accuracy:", best_accuracy)
print("Best Hyperparameters:", best_params)

Best Accuracy: 0.879800021648407
Best Hyperparameters: {'hidden_layer_size': 128, 'dropout_rate': 0.2, 'learning_rate': 0.001}
